<a href="https://colab.research.google.com/github/Saeeeae/Pytorch-Studying/blob/master/GAN_MNIST_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pickle
from google.colab import drive

drive.mount('/content/gdrive')
# "gdrive/My Drive/Colab Notebooks/faces/face_landmarks.csv"

# data 전처리 방식 지정
transform = transforms.Compose([
                                transforms.ToTensor(),  # data를 pytorch Tensor 형식으로 변형
                                transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

mnist = datasets.MNIST(root='gdrive/My Drive/Colab Notebooks/data', download=True, transform=transform)

dataloader = DataLoader(mnist, batch_size=60, shuffle=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
import imageio

use_gpu = torch.cuda.is_available()

leave_log = True

if leave_log:
  result_dir = 'gdrive/My Drive/Colab Notebooks/data/GAN_generated_images'
  if not os.path.isdir(result_dir):
    os.mkdir(result_dir)

In [0]:
### GAN 생성자 (Generator)
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지 출력함

class Generator(nn.Module):

  # 네트워크 구조
  def __init__(self):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
        nn.Linear(in_features=100, out_features=256),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Linear(in_features=256, out_features=512),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Linear(in_features=512, out_features=1024),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Linear(in_features=1024, out_features=28*28),
        nn.Tanh()
    )

    # batch_size * 100 크기의 랜덤 벡터를 받아
    # 이미지를 batch_size * 1 * 28 * 28 크기로 출력함
  def forward(self, inputs):
    return self.main(inputs).view(-1,1,28,28)

In [0]:
### GAN 구분자 (Discriminator)
# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력
class Discriminator(nn.Module):

  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
        nn.Linear(in_features=28*28, out_features=1024),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Dropout(inplace=False),
        nn.Linear(in_features=1024, out_features=512),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Dropout(inplace=False),
        nn.Linear(in_features=512, out_features=256),
        nn.LeakyReLU(0.2, inplace=False),
        nn.Dropout(inplace=False),
        nn.Linear(in_features=256, out_features=1),
        nn.Sigmoid()
    )

  # batch_size * 1 * 28 * 28 크기 이미지를 받아
  # 이미지가 진짜일 확률을 0 ~ 1 사이로 출력
  def forward(self, inputs):
    inputs = inputs.view(-1, 28*28)
    return self.main(inputs)

In [0]:
### 생성자와 구분자 객체 만들기

G = Generator()
D = Discriminator()

if use_gpu:
  G.cuda()
  D.cuda()


In [0]:
### 손실함수와 최적화 기법 지정
# Binary Cross Entropy Loss
criterion = nn.BCELoss()

# 생성자 매개변수를 최적화 하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas= (0.5, 0.999))
# 구분자의 매개변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [0]:
# 학습결과 시각화
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
  if type(data) == list:
    data = np.concatenate(data)
  # normalize data for display
  data = (data - data.min()) / (data.max() - data.min())

  # force the number of filters to be square
  n = int(np.ceil(np.sqrt(data.shape[0])))
  ### ???

  padding = (((0, n**2 - data.shape[0]), (0, 1), (0, 1))+ ((0, 0),) * (data.ndim -3))
  ## padding 이해해보기 !!!!

  data = np.pad(data, padding, mode='constant', constant_values=1)

  data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))

  data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])

  plt.imsave(path, data, cmap='gray')

In [0]:
if leave_log:
  train_hist = {}
  train_hist['D_losses'] = []
  train_hist['G_losses'] = []
  generated_images = []

z_fixed = Variable(torch.randn(5*5, 100), volatile = True)

if use_gpu:
  z_fixed = z_fixed.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


In [0]:
for epoch in range(100):
  
  if leave_log:
    D_losses = []
    G_losses = []

  # 한번에 batch_size 만큼 데이터를 가져옴
  for real_data, _ in dataloader:
    batch_size = real_data.size(0)

    # 데이터를 pytorch 변수로 변환
    real_data = Variable(real_data)

    ### 구분자 학습

    # 이미지가 진짜일 때 정답 값은 1, 가짜 값은 0
    # 정답지에 해당하는 변수를 만듦
    target_real = Variable(torch.ones(batch_size, 1))
    target_fake = Variable(torch.zeros(batch_size, 1))

    if use_gpu:
      real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()

    # 진짜 이미지를 구분자에 넣음
    D_result_from_real = D(real_data)
    # 구분자 출력값이 정답지인 1에서 멀수록 loss가 높아짐
    D_loss_real = criterion(D_result_from_real, target_real)

    # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다
    z = Variable(torch.randn((batch_size, 100)))

    if use_gpu:
      z = z.cuda()

    # 생성자로 가짜 이미지를 생성
    fake_data = G(z)

    # 생성자가 만든 가짜 이미지를 구분자에 넣음
    D_result_from_fake = D(fake_data)
    # 구분자 출력값이 정답지인 0에서 멀수록 loss가 높음
    D_loss_fake = criterion(D_result_from_fake, target_fake)

    # 구분자의 loss는 두 문제에서 계산된 loss의 합
    D_loss = D_loss_real + D_loss_fake

    # 구분자 매개변수 미분값을 0으로 초기화
    D.zero_grad()
    # 역전파를 통해 매개변수의 loss에 대한 미분값 계산
    D_loss.backward()
    # 최적화 기법을 이용해 구분자 매개변수 업데이트
    D_optimizer.step()

    if leave_log:
      D_losses.append(D_loss.item())

      # train generator G
      ### 생성자 학습시키기

      # 생성자에 입력으로 줄 랜덤 벡터 z를 만듦
      z = Variable(torch.randn((batch_size, 100)))

      if use_gpu:
        z = z.cuda()

      # 생성자로 가짜 이미지를 생성
      fake_data = G(z)
      # 생성자가 만든 가짜 이미지를 구분자에 넣음
      D_result_from_fake = D(fake_data)
      # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아짐
      G_loss = criterion(D_result_from_fake, target_real)

      # 생성자의 매개 변수 미분값을 0으로 초기화
      G.zero_grad()
      # 역전파를 통해 매개변수 loss에 대한 미분값 계산
      G_loss.backward()
      # 최적화 기법을 통해 생성자 매개변수 업데이트
      G_optimizer.step()

      if leave_log:
        G_losses.append(G_loss.item())
    
  if leave_log:
    true_positive_rate = (D_result_from_real > 0.5).float().mean().data
    true_negative_rate = (D_result_from_fake < 0.5).float().mean().data
    base_message = ("Epoch: {epoch:<3d} D Loss: {d_loss:<8.6} G Loss: {g_loss:<8.6} " "True Positive Rate: {tpr:<5.1%} True Negative Rate: {tnr:<5.1%}")

    message = base_message.format(
        epoch = epoch,
        d_loss = sum(D_losses)/len(D_losses),
        g_loss = sum(G_losses)/len(G_losses),
        tpr = true_positive_rate,
        tnr = true_negative_rate
    )
    print(message)
    
  if leave_log:
    fake_data_fixed = G(z_fixed)
    image_path = result_dir + '/epoch{}.png'.format(epoch)
    square_plot(fake_data_fixed.view(25, 28, 28).cpu().data.numpy(), path=image_path)
    generated_images.append(image_path)
    
  if leave_log:
    train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
    train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

torch.save(G.state_dict(), "gan_generator.pkl")  
torch.save(D.state_dict(), "gan_discriminator.pkl")
with open('gan_train_history.pkl', 'wb') as f:
  pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/GAN_generation.gif', generated_image_array, fps=5)

Epoch: 0   D Loss: 1.35443  G Loss: 0.702965 True Positive Rate: 68.3% True Negative Rate: 93.3%
Epoch: 1   D Loss: 1.35443  G Loss: 0.702967 True Positive Rate: 68.3% True Negative Rate: 91.7%
Epoch: 2   D Loss: 1.35451  G Loss: 0.703019 True Positive Rate: 80.0% True Negative Rate: 93.3%
Epoch: 3   D Loss: 1.35431  G Loss: 0.702968 True Positive Rate: 63.3% True Negative Rate: 91.7%
Epoch: 4   D Loss: 1.35452  G Loss: 0.702903 True Positive Rate: 81.7% True Negative Rate: 91.7%
Epoch: 5   D Loss: 1.35418  G Loss: 0.702937 True Positive Rate: 68.3% True Negative Rate: 98.3%
Epoch: 6   D Loss: 1.35437  G Loss: 0.702901 True Positive Rate: 60.0% True Negative Rate: 91.7%
Epoch: 7   D Loss: 1.35423  G Loss: 0.702936 True Positive Rate: 71.7% True Negative Rate: 93.3%
Epoch: 8   D Loss: 1.35427  G Loss: 0.702915 True Positive Rate: 60.0% True Negative Rate: 90.0%
Epoch: 9   D Loss: 1.35461  G Loss: 0.702893 True Positive Rate: 61.7% True Negative Rate: 86.7%
Epoch: 10  D Loss: 1.35415  G 

KeyboardInterrupt: ignored